# Efficientnetb0

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class conv_bn_act(nn.Module):
    def __init__(self, inchannels, outchannels, kernelsize, stride=1, dilation=1, groups=1, bias=False, bn_momentum=0.99):
        super().__init__()
        self.block = nn.Sequential(
            SameConv(inchannels, outchannels, kernelsize, stride, dilation, groups, bias=bias),
            nn.BatchNorm2d(outchannels, momentum=1-bn_momentum),
            swish()
        )

    def forward(self, x):
        return self.block(x)

class SameConv(nn.Conv2d):
    def __init__(self, inchannels, outchannels, kernelsize, stride=1, dilation=1, groups=1, bias=False):
        super().__init__(inchannels, outchannels, kernelsize, stride,
                         padding=0, dilation=dilation, groups=groups, bias=bias)

    def how_padding(self, n, kernel, stride, dilation):
        out_size = (n + stride - 1) // stride
        real_kernel = (kernel - 1) * dilation + 1
        padding_needed = max(0, (out_size - 1) * stride + real_kernel - n)
        is_odd = padding_needed % 2
        return padding_needed, is_odd

    def forward(self, x):
        row_padding_needed, row_is_odd = self.how_padding(x.size(2), self.weight.size(2), self.stride[0], self.dilation[0])
        col_padding_needed, col_is_odd = self.how_padding(x.size(3), self.weight.size(3), self.stride[1], self.dilation[1])
        if row_is_odd or col_is_odd:
            x = F.pad(x, [0, col_is_odd, 0, row_is_odd])

        return F.conv2d(x, self.weight, self.bias, self.stride,
                        (row_padding_needed//2, col_padding_needed//2), self.dilation, self.groups)

class swish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)


class SE(nn.Module):
    def __init__(self, inchannels, mid):
        super().__init__()
        self.AvgPool = nn.AdaptiveAvgPool2d(1)
        self.SEblock = nn.Sequential(
            nn.Linear(inchannels, mid),
            swish(),
            nn.Linear(mid, inchannels)
        )

    def forward(self, x):
        out = self.AvgPool(x)
        out = out.view(x.size(0), -1)
        out = self.SEblock(out)
        out = out.view(x.size(0), x.size(1), 1, 1)
        return x * torch.sigmoid(out)


class drop_connect(nn.Module):
    def __init__(self, survival=0.8):
        super().__init__()
        self.survival = survival

    def forward(self, x):
        if not self.training:
            return x

        random = torch.rand((x.size(0), 1, 1, 1), device=x.device)
        random += self.survival
        random.requires_grad = False
        return x / self.survival * torch.floor(random)


In [ ]:
import torch.nn as nn
import math

class MBConv(nn.Module):
    def __init__(self, inchannels, outchannels, expan, kernelsize, stride, se_ratio=4,
                 is_skip=True, dc_ratio=(1-0.8), bn_momentum=0.90):
        super().__init__()
        mid = expan * inchannels
        self.pointwise1 = conv_bn_act(inchannels, mid, 1) if expan != 1 else nn.Identity()
        self.depthwise = conv_bn_act(mid, mid, kernelsize, stride=stride, groups=mid)
        self.se = SE(mid, int(inchannels/se_ratio))
        self.pointwise2 = nn.Sequential(
            SameConv(mid, outchannels, 1),
            nn.BatchNorm2d(outchannels, 1-bn_momentum)
        )
        self.skip = is_skip and inchannels == outchannels and stride == 1
        self.dc = nn.Identity()

    def forward(self, x):
        residual = self.pointwise1(x)
        residual = self.depthwise(residual)
        residual = self.se(residual)
        residual = self.pointwise2(residual)
        if self.skip:
            residual = self.dc(residual)
            out = residual + x
        else:
            out = residual

        return out


class MBblock(nn.Module):
    def __init__(self, inchannels, outchannels, expan, kernelsize, stride, se_ratio, repeat,
                 is_skip, dc_ratio=(1-0.8), bn_momentum=0.90):
        super().__init__()

        layers = []
        layers.append(MBConv(inchannels, outchannels, expan, kernelsize, stride,
                             se_ratio, is_skip, dc_ratio, bn_momentum))
        while repeat-1:
            layers.append(MBConv(outchannels, outchannels, expan, kernelsize, 1,
                                 se_ratio, is_skip, dc_ratio, bn_momentum))
            repeat = repeat - 1

        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)


class EfficientNet(nn.Module):
    def __init__(self, width_multipler, depth_multipler, do_ratio, min_width=0, width_divisor=8,
                 se_ratio=4, dc_ratio=(1-0.8), bn_momentum=0.90, num_class=100):
        super().__init__()

        def renew_width(x):
            min = max(min_width, width_divisor)
            x *= width_multipler
            new_x = max(min, int((x + width_divisor/2) // width_divisor * width_divisor))

            if new_x < 0.9 * x:
                new_x += width_divisor
            return int(new_x)

        def renew_depth(x):
            return int(math.ceil(x * depth_multipler))

        self.stage1 = nn.Sequential(
            SameConv(3, renew_width(32), 3),
            nn.BatchNorm2d(renew_width(32), momentum=bn_momentum),
            swish()
        )
        self.stage2 = nn.Sequential(
                    # inchannels     outchannels  expand k  s(mobilenetv2)  repeat      is_skip
            MBblock(renew_width(32), renew_width(16), 1, 3, 1, se_ratio, renew_depth(1), True, dc_ratio, bn_momentum),
            MBblock(renew_width(16), renew_width(24), 6, 3, 2, se_ratio, renew_depth(2), True, dc_ratio, bn_momentum),
            MBblock(renew_width(24), renew_width(40), 6, 5, 2, se_ratio, renew_depth(2), True, dc_ratio, bn_momentum),
            MBblock(renew_width(40), renew_width(80), 6, 3, 2, se_ratio, renew_depth(3), True, dc_ratio, bn_momentum),
            MBblock(renew_width(80), renew_width(112), 6, 5, 1, se_ratio, renew_depth(3), True, dc_ratio, bn_momentum),
            MBblock(renew_width(112), renew_width(192), 6, 5, 1, se_ratio, renew_depth(4), True, dc_ratio, bn_momentum),
            MBblock(renew_width(192), renew_width(320), 6, 3, 1, se_ratio, renew_depth(1), True, dc_ratio, bn_momentum)
        )
        self.stage3 = nn.Sequential(
            SameConv(renew_width(320), renew_width(1280), 1, stride=1),
            nn.BatchNorm2d(renew_width(1280), bn_momentum),
            swish(),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(do_ratio)
        )
        self.FC = nn.Linear(renew_width(1280), num_class)
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, SameConv):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
            elif isinstance(m, nn.Linear):
                bound = 1/int(math.sqrt(m.weight.size(1)))
                nn.init.uniform(m.weight, -bound, bound)

    def forward(self, x):
        out = self.stage1(x)
        out = self.stage2(out)
        out = self.stage3(out)
        out = out.view(out.size(0), -1)
        out = self.FC(out)
        return out


def efficientnet(width_multipler, depth_multipler, num_class=100, bn_momentum=0.90, do_ratio=0.2):
    return EfficientNet(width_multipler, depth_multipler,
                        num_class=num_class, bn_momentum=bn_momentum, do_ratio=do_ratio)

# Training

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Subset

# Define the data transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Download the CIFAR-100 dataset
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)

# Split the dataset into training, validation, and test sets
train_size = int(0.8 * len(train_dataset))
val_size = int(0.1 * len(train_dataset))
test_size = len(train_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(train_dataset, [train_size, val_size, test_size])

# Create balanced DataLoader for training, validation, and test sets
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
!git clone https://github.com/K-Hooshanfar/small-net-cifar100

Cloning into 'small-net-cifar100'...
remote: Enumerating objects: 487, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 487 (delta 0), reused 0 (delta 0), pack-reused 484
Receiving objects: 100% (487/487), 186.82 MiB | 15.91 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Updating files: 100% (424/424), done.


In [ ]:
%cd small-net-cifar100

/content/small-net-cifar100


In [ ]:
!python train.py -net efficientnetb0

Streaming output truncated to the last 5000 lines.
Epoch:84	 Step:341	 TrainedSample:43648	 TotalSample:50000	 Loss:1.206
Epoch:84	 Step:351	 TrainedSample:44928	 TotalSample:50000	 Loss:1.462
Epoch:84	 Step:361	 TrainedSample:46208	 TotalSample:50000	 Loss:1.826
Epoch:84	 Step:371	 TrainedSample:47488	 TotalSample:50000	 Loss:1.274
Epoch:84	 Step:381	 TrainedSample:48768	 TotalSample:50000	 Loss:1.175
Epoch:84	 Step:391	 TrainedSample:50000	 TotalSample:50000	 Loss:1.188
evaluating
saving regular
saving best
Epoch:85	 Step:1	 TrainedSample:128	 TotalSample:50000	 Loss:1.342
Epoch:85	 Step:11	 TrainedSample:1408	 TotalSample:50000	 Loss:1.181
Epoch:85	 Step:21	 TrainedSample:2688	 TotalSample:50000	 Loss:1.217
Epoch:85	 Step:31	 TrainedSample:3968	 TotalSample:50000	 Loss:1.321
Epoch:85	 Step:41	 TrainedSample:5248	 TotalSample:50000	 Loss:1.164
Epoch:85	 Step:51	 TrainedSample:6528	 TotalSample:50000	 Loss:1.139
Epoch:85	 Step:61	 TrainedSample:7808	 TotalSample:50000	 Loss:1.321
Epoc

# Save weights

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import shutil

source_path = '/content/small-net-cifar100/checkpoint/efficientnetb0/2023-12-02/bestParam.pth'
destination_path = '/content/gdrive/My Drive/bestParam.pth'

shutil.copyfile(source_path, destination_path)


'/content/gdrive/My Drive/bestParam.pth'

# load model

In [ ]:
import torch

# Define the EfficientNet model
model = efficientnet(1, 1, 100, bn_momentum=0.9)

# Load pre-trained weights
pretrained_weights_path = '/content/small-net-cifar100/checkpoint/efficientnetb0/2023-12-02/bestParam.pth'
state_dict = torch.load(pretrained_weights_path)

# Load the state_dict into the model
model.load_state_dict(state_dict)


<ipython-input-2-2b031e1ed132>:106: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(m.weight, -bound, bound)


<All keys matched successfully>

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

# Define data transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-100 dataset
cifar100_test = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)


# Create PyTorch data loaders
batch_size = 128
val_loader = torch.utils.data.DataLoader(cifar100_test, batch_size=batch_size, shuffle=False)

# Check the number of samples in each set
print(f"Test set size: {len(cifar100_test)}")

Files already downloaded and verified
Test set size: 10000


In [2]:
model.eval()
correct_test = 0
total_test = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f"Final Test Accuracy: {test_accuracy:.2f}%")


Final Test Accuracy: 69.78%


# Load Model and Get Feature Latent

In [15]:
from torch.utils.data.sampler import SubsetRandomSampler
import torch
from torchvision import datasets, transforms
import random
import numpy as np

# Set random seeds for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

# Load the CIFAR-100 dataset and create a balanced subset
transform = transforms.Compose([transforms.ToTensor()])
# Load CIFAR-100 dataset
cifar100_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(cifar100_dataset))
val_size = len(cifar100_dataset) - train_size
cifar100_traindataset, cifar100_valdataset = torch.utils.data.random_split(cifar100_dataset, [train_size, val_size])

# Define the subset size
subset_fraction = 1
subset_size_train = int(subset_fraction * len(cifar100_traindataset))

class_indices = list(range(len(cifar100_traindataset.dataset.classes)))
class_subset_size = int(subset_size_train / len(cifar100_traindataset.dataset.classes))

class_sampler_indices_train = []

for class_index in class_indices:
    class_indices_list_train = [i for i, label in enumerate(cifar100_traindataset.dataset.targets) if label == class_index]
    class_sampler_indices_train.extend(class_indices_list_train[:class_subset_size])


print(f"Length of class_sampler_indices_train: {len(class_sampler_indices_train)}")

train_sampler = SubsetRandomSampler(class_sampler_indices_train)

batch_size = 256
train_loader = torch.utils.data.DataLoader(cifar100_traindataset, batch_size=batch_size, sampler=train_sampler)

# Check the number of samples in the balanced train set
print(f"Balanced Train set size: {len(train_loader.sampler)}")

Files already downloaded and verified
Length of class_sampler_indices_train: 8000
Balanced Train set size: 8000


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the EfficientNet model
model = efficientnet(1, 1, 100, bn_momentum=0.9)

# Load pre-trained weights
pretrained_weights_path = '/content/small-net-cifar100/checkpoint/efficientnetb0/2023-12-02/bestParam.pth'
state_dict = torch.load(pretrained_weights_path)

# Load the state_dict into the model
model.load_state_dict(state_dict)

model = nn.Sequential(*list(model.children())[:-1]).to(device)

# Initialize lists to store data and labels
data_list = []
labels_list = []

# Iterate through the balanced training set to extract data and labels
for inputs, labels in train_loader:
    if torch.cuda.is_available():
        inputs = inputs.to('cuda')
    # Forward pass through the model to get feature representation
    features = model(inputs)

    # Append the features and labels to the lists
    data_list.append(features)
    labels_list.append(labels)

    # Release GPU memory
    del inputs
    torch.cuda.empty_cache()


<ipython-input-2-2b031e1ed132>:106: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(m.weight, -bound, bound)


In [ ]:
# Stack and reshape the extracted features
features = torch.cat(data_list)
features = features.view(features.size(0), -1)
labels = torch.cat(labels_list)
labels = labels.unsqueeze(1)

# Metrics for train set

In [2]:
!git clone https://github.com/Arhosseini77/data_complexity_measures

Cloning into 'data_complexity_measures'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 140 (delta 80), reused 33 (delta 9), pack-reused 0
Receiving objects: 100% (140/140), 144.00 KiB | 1.40 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from torch.utils.data import Subset, DataLoader
import numpy as np
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data.sampler import SubsetRandomSampler

from data_complexity_measures.models.ARH_SeparationIndex import ARH_SeparationIndex

In [ ]:
# Create Instance of class
si_calculator = ARH_SeparationIndex(features, labels, normalize=True)

Data has been normalized


# SI

In [ ]:
si_data = si_calculator.si_batch(batch_size=1000)
print(si_data)

Calculating SI: 100%|██████████| 1/1 [00:00<00:00, 121.46it/s]

0.4624999761581421


# Calc High order SI (order = 2)

In [ ]:
si_high_order_2_data = si_calculator.high_order_si_batch(order=2,batch_size=1000)
print(si_high_order_2_data)

Computing High Order SI: 100%|██████████| 1/1 [00:01<00:00,  1.68s/it]

0.2524999976158142


# High order soft SI (order=2)

In [ ]:
si_soft_order_2_data = si_calculator.soft_order_si_batch(order=2,batch_size=1000)
print("Soft Order(2) SI :", si_soft_order_2_data)

Calculating Soft Order SI: 100%|██████████| 1/1 [00:00<00:00, 727.93it/s]

Soft Order(2) SI : 0.4387499988079071


# Center Based SI

In [1]:
center_si_data = si_calculator.center_si_batch(batch_size=1000)
print("Center SI:", center_si_data)

Center SI: 0.681


# Anti SI (order = 2)

In [ ]:
anti_si = si_calculator.anti_si(order=2)
print("anti_si:", anti_si)

Calculating Anti-SI: 100%|██████████| 400/400 [00:00<00:00, 10979.64it/s]

anti_si: 0.375


# Metrics for test set

In [19]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from torch.utils.data.sampler import SubsetRandomSampler

# Load the CIFAR-100 dataset and create a balanced subset
transform = transforms.Compose([transforms.ToTensor()])
cifar100_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

# Define the subset size
subset_fraction = 1
subset_size_test = int(subset_fraction * len(cifar100_dataset))

# Create a balanced subset for both train and test sets using SubsetRandomSampler
class_indices = list(range(len(cifar100_dataset.classes)))
class_subset_size = int(subset_size_test / len(cifar100_dataset.classes))

class_sampler_indices_test = []

for class_index in class_indices:
    class_indices_list_train = [i for i, label in enumerate(cifar100_dataset.targets) if label == class_index]
    class_sampler_indices_test.extend(class_indices_list_train[:class_subset_size])

test_sampler = SubsetRandomSampler(class_sampler_indices_test)

# Create PyTorch data loaders using the balanced subset for both train and test sets
batch_size = 256
test_loader = torch.utils.data.DataLoader(cifar100_dataset, batch_size=batch_size, sampler=test_sampler)

# Check the number of samples in each set
print(f"Balanced Train set size: {len(test_loader.sampler)}")

Files already downloaded and verified
Balanced Train set size: 10000


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the EfficientNet model
model = efficientnet(1, 1, 100, bn_momentum=0.9)

# Load pre-trained weights
pretrained_weights_path = '/content/small-net-cifar100/checkpoint/efficientnetb0/2023-12-02/bestParam.pth'
state_dict = torch.load(pretrained_weights_path)

# Load the state_dict into the model
model.load_state_dict(state_dict)

model = nn.Sequential(*list(model.children())[:-1]).to(device)

# Initialize lists to store data and labels
data_list = []
labels_list = []

# Iterate through the balanced training set to extract data and labels
for inputs, labels in test_loader:
    if torch.cuda.is_available():
        inputs = inputs.to('cuda')
    # Forward pass through the model to get feature representation
    features = model(inputs)

    # Append the features and labels to the lists
    data_list.append(features)
    labels_list.append(labels)

    # Release GPU memory
    del inputs
    torch.cuda.empty_cache()


<ipython-input-4-2b031e1ed132>:106: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(m.weight, -bound, bound)


In [ ]:
# Stack and reshape the extracted features
features = torch.cat(data_list)
features = features.view(features.size(0), -1)
labels = torch.cat(labels_list)
labels = labels.unsqueeze(1)

In [ ]:
# Create Instance of class
si_calculator = ARH_SeparationIndex(features, labels, normalize=True)

Data has been normalized


# SI

In [ ]:
si_data = si_calculator.si_batch(batch_size=1000)
print(si_data)

Calculating SI: 100%|██████████| 1/1 [00:00<00:00, 1040.25it/s]

0.36500000953674316


# Calc High order SI (order = 2)

In [ ]:
si_high_order_2_data = si_calculator.high_order_si_batch(order=2,batch_size=1000)
print(si_high_order_2_data)

Computing High Order SI: 100%|██████████| 1/1 [00:00<00:00, 210.73it/s]

0.18000000715255737


# High order soft SI (order=2)

In [ ]:
si_soft_order_2_data = si_calculator.soft_order_si_batch(order=2,batch_size=1000)
print("Soft Order(2) SI :", si_soft_order_2_data)

Calculating Soft Order SI: 100%|██████████| 1/1 [00:00<00:00, 582.22it/s]

Soft Order(2) SI : 0.31950002908706665


# Center Based SI

In [ ]:
center_si_data = si_calculator.center_si_batch(batch_size=1000)
print("Center SI:", center_si_data)

Calculating CSI: 100%|██████████| 1/1 [00:00<00:00, 741.57it/s]

Center SI: 0.9270000457763672


# Anti SI (order = 2)

In [ ]:
anti_si = si_calculator.anti_si(order=2)
print("anti_si:", anti_si)

Calculating Anti-SI: 100%|██████████| 1000/1000 [00:00<00:00, 14613.43it/s]

anti_si: 0.5410000085830688


# Pretrained Efficientnetb0

In [4]:
!pip install -qq efficientnet_pytorch

In [3]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')

Loaded pretrained weights for efficientnet-b0


In [4]:
model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [16]:
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def extract_features(model, dataloader):
    model.eval()
    model.to(device)

    # Extract features up to _avg_pooling
    features = []
    labels = []

    # Switch off gradient computation
    with torch.no_grad():
        for images, targets in tqdm(dataloader):
            images, targets = images.to(device), targets.to(device)

            # Forward pass up to _avg_pooling
            x = model._conv_stem(images)
            x = model._bn0(x)
            for block in model._blocks:
                x = block(x)
                if isinstance(block, models.efficientnet.SqueezeExcitation):
                    # SqueezeExcitation block, apply Swish activation
                    x = block._swish(x)
            x = model._conv_head(x)
            x = model._bn1(x)
            x = model._swish(x)
            x = model._avg_pooling(x)

            # Flatten the output before appending to features
            features.append(x.view(x.size(0), -1))
            labels.append(targets)

    features = torch.cat(features)
    labels = torch.cat(labels)

    return features, labels

# Move models to GPU
model.to(device)

efficientnet_features, efficientnet_labels = extract_features(model, train_loader)

100%|██████████| 32/32 [00:01<00:00, 19.25it/s]


In [8]:
efficientnet_features.shape

torch.Size([32000, 1280])

In [17]:
si_calculator = ARH_SeparationIndex(efficientnet_features, efficientnet_labels, normalize=True)

Data has been normalized


**Train**


# SI

In [10]:
si_data = si_calculator.si_batch(batch_size=1000)
print(si_data)

Calculating SI: 100%|██████████| 32/32 [00:00<00:00, 5791.49it/s]

0.12784375250339508


# Calc High order SI (order = 2)

In [11]:
si_high_order_2_data = si_calculator.high_order_si_batch(order=2,batch_size=1000)
print(si_high_order_2_data)

Computing High Order SI: 100%|██████████| 32/32 [00:00<00:00, 382.11it/s]


0.039750002324581146


# High order soft SI (order=2)

In [12]:
si_soft_order_2_data = si_calculator.soft_order_si_batch(order=2,batch_size=1000)
print("Soft Order(2) SI :", si_soft_order_2_data)

Calculating Soft Order SI: 100%|██████████| 32/32 [00:00<00:00, 1671.35it/s]


Soft Order(2) SI : 0.11173438280820847


# Center Based SI

In [13]:
center_si_data = si_calculator.center_si_batch(batch_size=1000)
print("Center SI:", center_si_data)

Calculating CSI: 100%|██████████| 32/32 [00:00<00:00, 3102.58it/s]

Center SI: 0.10343750566244125


# Anti SI (order = 2)

In [18]:
anti_si = si_calculator.anti_si(order=2)
print("anti_si:", anti_si)

Calculating Anti-SI: 100%|██████████| 8000/8000 [00:00<00:00, 10959.27it/s]

anti_si: 0.8500000238418579


**Test**

In [20]:
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def extract_features(model, dataloader):
    model.eval()
    model.to(device)

    # Extract features up to _avg_pooling
    features = []
    labels = []

    # Switch off gradient computation
    with torch.no_grad():
        for images, targets in tqdm(dataloader):
            images, targets = images.to(device), targets.to(device)

            # Forward pass up to _avg_pooling
            x = model._conv_stem(images)
            x = model._bn0(x)
            for block in model._blocks:
                x = block(x)
                if isinstance(block, models.efficientnet.SqueezeExcitation):
                    # SqueezeExcitation block, apply Swish activation
                    x = block._swish(x)
            x = model._conv_head(x)
            x = model._bn1(x)
            x = model._swish(x)
            x = model._avg_pooling(x)

            # Flatten the output before appending to features
            features.append(x.view(x.size(0), -1))
            labels.append(targets)

    features = torch.cat(features)
    labels = torch.cat(labels)

    return features, labels

# Move models to GPU
efficientnet_features.to(device)

efficientnet_features, efficientnet_labels = extract_features(model, test_loader)

100%|██████████| 40/40 [00:02<00:00, 17.13it/s]


In [21]:
si_calculator = ARH_SeparationIndex(efficientnet_features, efficientnet_labels, normalize=True)

Data has been normalized


# SI

In [22]:
si_data = si_calculator.si_batch(batch_size=1000)
print(si_data)

Calculating SI: 100%|██████████| 10/10 [00:00<00:00, 3928.72it/s]

0.1022999957203865


# Calc High order SI (order = 2)

In [23]:
si_high_order_2_data = si_calculator.high_order_si_batch(order=2,batch_size=1000)
print(si_high_order_2_data)

Computing High Order SI: 100%|██████████| 10/10 [00:00<00:00, 1083.86it/s]


0.0284000001847744


# High order soft SI (order=2)

In [24]:
si_soft_order_2_data = si_calculator.soft_order_si_batch(order=2,batch_size=1000)
print("Soft Order(2) SI :", si_soft_order_2_data)

Calculating Soft Order SI: 100%|██████████| 10/10 [00:00<00:00, 946.15it/s]


Soft Order(2) SI : 0.08899999409914017


# Center Based SI

In [25]:
center_si_data = si_calculator.center_si_batch(batch_size=1000)
print("Center SI:", center_si_data)

Calculating CSI: 100%|██████████| 10/10 [00:00<00:00, 1102.37it/s]

Center SI: 0.11729999631643295


# Anti SI (order = 2)

In [26]:
anti_si = si_calculator.anti_si(order=2)
print("anti_si:", anti_si)

Calculating Anti-SI: 100%|██████████| 10000/10000 [00:01<00:00, 9023.23it/s]

anti_si: 0.850600004196167
